Reload modules every time before executing the Python code typed:

In [1]:
%load_ext autoreload
%autoreload 2

Install necessary packages:

In [2]:
%pip install opencv-python numpy scipy filterpy==1.4.5

Note: you may need to restart the kernel to use updated packages.


# Task 1 (exercise 1.2)

Import necessary packages:

In [17]:
import cv2
import numpy as np
from scipy.stats import linregress

import time
import sys
sys.path.append('./')

from sort import Sort

The higher the value, the bigger a moving object should be to be detected

In [4]:
MIN_CONTOUR_AREA = 4000

In [19]:
class Tracker:
    class TrackingObject:
        def __init__(self, contour, tracking_id, ttl) -> None:
            self.contour = contour
            self.centroid = Tracker.calc_centroid(contour)
            self.tracking_id = tracking_id
            self.ttl = ttl
            

    def __init__(self, proximity_threshold = 20, object_ttl = 10) -> None:
        self.objects = []
        self.last_tracking_id = 0
        self.proximity_threshold = proximity_threshold
        self.object_ttl = object_ttl

    @classmethod
    def calc_centroid(cls, contour):
        """
        Calculates the centroid of a contour.
        """
        M = cv2.moments(contour)
        if M["m00"] == 0:
            # Avoid division by zero
            return None

        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])

        return (cX, cY)

    def update(self, contours):
        """
        Updates the tracker with a new frame information: contours and centroids.
        """
        original_contours = contours.copy()
        contours = contours.copy()

        # Find the closest object to each contour centroid
        for i, contour in enumerate(contours):
            # Find the centroid of each contour
            centroid = Tracker.calc_centroid(contour)

            if centroid is None:
                continue

            # Find the closest object to the centroid
            closest_object = None
            closest_distance = float('inf')
            for obj in self.objects:
                distance = np.linalg.norm(np.array(obj.centroid) - np.array(centroid))
                if distance < closest_distance:
                    closest_object = obj
                    closest_distance = distance

            # If the closest object is close enough, update it
            if closest_distance < self.proximity_threshold:
                closest_object.centroid = centroid
                # Remove the centroid from the list
                contours[i] = None
        
        # Add the remaining contours as new objects
        for contour in contours:
            if contour is None:
                continue

            for object in self.objects:
                # If the contour is already have any object, skip it
                if cv2.pointPolygonTest(contour, object.centroid, False) > 0:
                    break
            else:
                # Please note that the first object will have ID 1 will be the whole frame
                self.last_tracking_id = self.last_tracking_id + 1
                print(f"Create a new object with ID: {self.last_tracking_id}")
                self.objects.append(Tracker.TrackingObject(contour, self.last_tracking_id, self.object_ttl))

        # Remove objects that are not covered by any contour
        for obj in self.objects:
            for contour in original_contours:
                if cv2.pointPolygonTest(contour, obj.centroid, False) > 0:
                    break
            else:
                obj.ttl -= 1
                if obj.ttl <= 0:
                    print(f"Remove object {obj.tracking_id}")
                    self.objects.remove(obj)


In [20]:
# Open the sample video file:
video = cv2.VideoCapture("Traffic_Laramie_1.mp4")

# Ensure that the file was opened correctly:
assert video.isOpened(), "Can't open the video file"

# Initialize an instance of [Gaussian Mixture-based Background/Foreground Segmentation Algorithm](https://docs.opencv.org/3.4/d7/d7b/classcv_1_1BackgroundSubtractorMOG2.html):
bg_subtractor = cv2.createBackgroundSubtractorKNN(detectShadows=False, history=10000, dist2Threshold=800)

# Instantiate the tracker
tracker = Tracker()

cv2.namedWindow("Cam")

counted_car_ids = set()

# Read the video frame by frame:
while True:
    ret, frame = video.read()

    # Check for the last frame
    if not ret:
        break

    # blur the frame to remove noise
    blured_frame = cv2.GaussianBlur(frame, (5,5), 0)

    # Apply the background subtraction algorithm:
    fg_mask = bg_subtractor.apply(blured_frame)

    # Apply morphological operations to remove corruptions:
    _, thresh = cv2.threshold(fg_mask, 25, 255, cv2.THRESH_BINARY)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)

    # Find the contours of the detected objects:
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter out the contours that are too small to be a car:
    contours = [c for c in contours if cv2.contourArea(c) > MIN_CONTOUR_AREA]

    # Convex hull to get clean contours without holes:
    contours = [cv2.convexHull(c) for c in contours]

    # Draw the contours on the original image:
    cv2.drawContours(frame, contours, -1, (0, 255, 0), 2)

    # Draw tracking objects:
    for obj in tracker.objects:
        cv2.putText(frame, f"Car #{obj.tracking_id}", obj.centroid, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        cv2.circle(frame, obj.centroid, 2, (0, 0, 255), -1)

    # Update the tracker with contours and centroids:
    tracker.update(contours)

    # Display the resulting frame
    cv2.imshow("Cam", frame)

    # Slow down the video for debugging purposes 
    # time.sleep(0.1)

    # Press "q" to exit the loop
    if (cv2.waitKey(1) & 0xFF == ord('q')):
        cv2.destroyWindow("Cam")
        cv2.waitKey(1)
        break

Create a new object with ID: 1
Remove object 1
Create a new object with ID: 2
Create a new object with ID: 3
Create a new object with ID: 4
Remove object 4
Create a new object with ID: 5
Remove object 5
